## Construcción de bases de datos y Análisis Exploratorio de los Datos

In [ ]:
import requests
import pandas as pd
import locale
import datetime
import os
import re
import py7zr
from io import StringIO, BytesIO
from functools import reduce
import eurostat
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.tseries.offsets import DateOffset

locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')  # Unix/Linux

'es_ES.UTF-8'

**Indicadores del modelo:**

* AFI: Trabajadores en alta laboral en la Seguridad Social. Media mensual. CVEC.
* PMIS: Índice PMI de Servicios.
* IPI: Índice de Producción Industrial. Índice General. CVEC.
* VGE: Grandes Empresas. Ventas totales deflactadas y CVEC.
* RBT: Grandes Empresas Retribución bruta deflactada y CVEC.
* IMPB: Importaciones de bienes total. Precios constantes. CVEC
* EPA: Ocupación EPA. CVEC.
* CNTR: Producto Interior Bruto. CVEC.

### Construcción de BBDD

A continuación, se detallan las funciones definidas para obtener los indicadores. En algunos casos, se necesita actualizar el enlace base para obtener la información.

#### AFI: Trabajadores en alta laboral en la Seguridad Social. Media mensual. CVEC.

Información obtenida de la Sede Electrónica de la Seguridad Social.

Se utiliza la serie de afiliados medios totales, disponible en el enlace: https://www.seg-social.es/wps/portal/wss/internet/EstadisticasPresupuestosEstudios/Estadisticas/EST8/EST10/EST290/EST291

El documento de interés es la "Serie de afiliación Media por regímenes 2001 - 2025".

No se indica de manera inequívoca que los datos sean CVEC, es decir, que hayan sido corregidos de las variaciones estacionales y del efecto calendario.

In [2]:
# Enlace: Se debe actualizar en cada actualización

url_afi = "https://www.seg-social.es/wps/wcm/connect/wss/5ccf558a-868f-48b3-b832-04fe9f524960/19_Serie+afiliaci%C3%B3n+media+por+reg%C3%ADmenes+2001-202504.xlsx?MOD=AJPERES&CONVERT_TO=linktext&CACHEID=ROOTWORKSPACE.Z18_81D21J401P5L40QTIT61G41000-5ccf558a-868f-48b3-b832-04fe9f524960-pqAlPXj"

In [ ]:
# Obtención de Trabajadores en alta laboral en la Seguridad Social. Media Mensual.

def obtener_afiliados_seguridad_social(url_afi):
    
    """Obtiene una serie temporal con el número de afiliados en alta laboral en la Seguridad Social española 
    (media mensual de todo el sistema).

    Parámetros:
    ----------
    url_afi : str
        Ruta local o URL del archivo Excel que contiene los datos de afiliación.

    Retorna:
    -------
    pd.DataFrame
        DataFrame con dos columnas:
            - 'Fecha': fechas en formato datetime (mensuales).
            - 'AFI': número de afiliados al sistema.
    """    
    
    df = pd.read_excel(url_afi, skiprows=1, usecols=['Régimen', 'TOTAL SISTEMA'])
    
    # Eliminar segunda línea vacía
    df = df.drop(axis=0, index=0)

    # Eliminar líneas sin valores (son notas)
    df = df.dropna(axis=0, how="any")

    # Pasar a formato fecha y eliminar columna
    df['Fecha'] = pd.to_datetime(df['Régimen'], format='%B %Y')
    df = df.drop(axis=1, columns="Régimen")
    
    # Cambio de orden
    series_afiliados = df[['Fecha', 'TOTAL SISTEMA']]

    series_afiliados = series_afiliados.rename(columns={'TOTAL SISTEMA': 'AFI'})

    return series_afiliados

In [4]:
# Obtener todas las series
serie_afiliados = obtener_afiliados_seguridad_social(url_afi)

In [5]:
serie_afiliados.head()

,Fecha,AFI
1,2001-01-01,15194299.22
2,2001-02-01,15326583.35
3,2001-03-01,15455386.40
4,2001-04-01,15551821.04
5,2001-05-01,15688072.27


In [6]:
serie_afiliados.tail()

,Fecha,AFI
288,2024-12-01,2.133796e+07
289,2025-01-01,2.109581e+07
290,2025-02-01,2.119615e+07
291,2025-03-01,2.135765e+07
292,2025-04-01,2.158864e+07


#### PMIS: Índice PMI de Servicios.

In [7]:
# Enlace: No es necesario actualizar
url_pmi = "https://www.mql5.com/en/economic-calendar/spain/markit-services-pmi/export"

In [ ]:
def obtener_series_PMI(url_pmi):
    
    """
    Descarga y procesa la serie temporal del índice PMI (Purchasing Managers' Index) de servicios en España.

    Parámetros:
    ----------
    url_pmi : str
        URL del archivo CSV (separado por tabulaciones) que contiene los valores del índice PMI.

    Retorna:
    -------
    pd.DataFrame
        DataFrame con dos columnas:
            - 'Fecha': primer día del mes correspondiente al valor PMI, en formato datetime.
            - 'PMI': valor del índice PMI convertido a número.
    """
    
    headers = {"User-Agent": "Mozilla/5.0"}

    # Descargar el archivo
    response = requests.get(url_pmi, headers=headers)
    response.raise_for_status()

    # Leer el CSV con separador TAB
    csv_text = response.content.decode("utf-8")
    df = pd.read_csv(StringIO(csv_text), sep='\t')

    # Renombrar columnas y convertir tipos
    df = df.rename(columns={
        "Date": "Fecha",
        "ActualValue": "PMI"
    })

    # Conversión de variables
    df["Fecha"] = pd.to_datetime(df["Fecha"], format="%Y.%m.%d") - pd.offsets.MonthBegin(1)
    df["PMI"] = pd.to_numeric(df["PMI"], errors='coerce')

    # Guardar fechas

    serie_pmi = df[["Fecha", "PMI"]].sort_values("Fecha")

    serie_pmi = serie_pmi.reset_index(drop=True)

    return serie_pmi


In [9]:
serie_pmi = obtener_series_PMI(url_pmi)

In [10]:
serie_pmi.head()

,Fecha,PMI
0,2011-05-01,50.4
1,2011-06-01,50.9
2,2011-07-01,50.2
3,2011-08-01,46.5
4,2011-09-01,45.2


In [11]:
serie_pmi.tail()

,Fecha,PMI
164,2025-01-01,57.3
165,2025-02-01,54.9
166,2025-03-01,56.2
167,2025-04-01,54.7
168,2025-05-01,53.4


#### Variables provenientes de la Agencia Estatal de Administración Tributaria

Se obtienen dos series de la Agencia Estatal de Administración Tributaria (AEAT):
* Grandes Empresas. Ventas totales deflactadas y CVEC.
* Grandes Empresas. Retribución bruta deflactada y CVEC.

Se disponibiliza un fichero .zip que contiene diversos ficheros:

* Vesge_pub_indexada: Fichero de texto que contiene los datos en su formato original (raw).
* Diseño de registro tablas aux: Documento auxiliar que describe la estructura y codificación del fichero original, y permite su correcta interpretación y conversión a series temporales analíticas

In [12]:
# Enlace: No es necesario actualizarlo

url_aeat = "https://sede.agenciatributaria.gob.es/static_files/AEAT/Estudios/Estadisticas/Informes_Estadisticos/Informe_VESGE/BdDVesge.7z"

In [ ]:
def obtener_series_AEAT(url_aeat):
    """
    Descarga, extrae y procesa las series económicas proporcionadas por la AEAT
    (Agencia Estatal de Administración Tributaria) de España.

    Parámetros:
    ----------
    url_aeat : str
        URL directa al archivo comprimido `.7z` que contiene los datos de la AEAT.

    Retorna:
    -------
    tuple:
        - serie_ventas (pd.DataFrame): Serie mensual del Valor de las Ventas en Grandes Empresas (VGE).
        - serie_rendimientos (pd.DataFrame): Serie mensual de Rendimientos Brutos del Trabajo (RBT).
    """

    filename = os.path.basename(url_aeat)

    # Función interna que realiza la consulta a la página de la AEAT y descarga el archivo

    def descargar_archivo():
        
        response = requests.get(url_aeat, stream=True)
        
        if response.status_code == 200:
            with open(filename, 'wb') as out:
                for chunk in response.iter_content(chunk_size=8192):
                    out.write(chunk)
            
            print("Descarga completada.")
        
        else:
            raise Exception(f"Error en la descarga: {response.status_code}")

    # Si el archivo ya existe
    if os.path.exists(filename):
        respuesta = input(f"El archivo '{filename}' ya existe. ¿Deseas sobrescribirlo? [s/n]: ").strip().lower()
        if respuesta == 's':
            print("Sobrescribiendo archivo existente...")
            descargar_archivo()
        else:
            print("Descarga cancelada. Se mantiene el archivo existente.")
    else:
        descargar_archivo()

    # Extracción del fichero .7z que contiene los ficheros de interés

    archivo_7z = "BdDVesge.7z"
    extract_dir = "BdDVesge_extract"
    os.makedirs(extract_dir, exist_ok=True)

    # Extraer con py7zr
    with py7zr.SevenZipFile(archivo_7z, mode='r') as archive:
        archive.extractall(path=extract_dir)

    # Mostrar todos los archivos extraídos
    for root, dirs, files in os.walk(extract_dir):
        for name in files:
            print("Archivo extraído:", os.path.join(root, name))

    # Buscar archivo .txt extraído (en caso de que el nombre varíe)
    txt_files = [f for f in os.listdir(extract_dir) if f.endswith('.txt')]
    print("Archivos .txt encontrados:", txt_files)

    # Ruta al fichero esperado
    ruta_txt = os.path.join(extract_dir, txt_files[0])  # si hay solo uno

    # Leer el contenido
    df = pd.read_csv(ruta_txt, sep=';', encoding='latin1')

    # Extracción de series 

    serie_ventas = df[(df["VARIABLE"] == 2) & (df["LITERAL"] == 29) & (df["FREC"] == 1) & (df["DATO1"] == 1) & (df["DATO2"] == 2)]

    serie_rendimientos = df[(df["VARIABLE"] == 11) & (df["LITERAL"] == 29) & (df["FREC"] == 1) & (df["DATO1"] == 1) & (df["DATO2"] == 2)]

    # Extracción de períodos

    serie_ventas = serie_ventas.reset_index(drop=True)
    serie_rendimientos = serie_rendimientos.reset_index(drop=True)

    # Para creación de columna Fecha:

    xlsx_files = [f for f in os.listdir(extract_dir) if f.endswith('.xlsx')]

    # Ruta al fichero esperado
    ruta_xlsx = os.path.join(extract_dir, xlsx_files[0])  # si hay solo uno

    # Leer el contenido
    df = pd.read_excel(ruta_xlsx, skiprows=3, usecols=[23, 24, 25])

    serie_ventas = serie_ventas.merge(df, left_on='PERIODO', right_on='Id_periodo', how='inner')
    serie_rendimientos = serie_rendimientos.merge(df, left_on='PERIODO', right_on='Id_periodo', how='inner')

    # Selección de variable

    serie_ventas = serie_ventas[['Literal', 'VALOR']].rename(columns={'VALOR': 'VGE', 'Literal':'Fecha'})
    serie_rendimientos = serie_rendimientos[['Literal', 'VALOR']].rename(columns={'VALOR': 'RBT', 'Literal':'Fecha'})

    # Convertir columna 'Literal' a datetime
    serie_ventas['Fecha'] = pd.to_datetime(serie_ventas['Fecha'], format="%Y-%m", errors='coerce') + pd.offsets.MonthBegin(0)
    serie_rendimientos['Fecha'] = pd.to_datetime(serie_rendimientos['Fecha'], format="%Y-%m", errors='coerce') + pd.offsets.MonthBegin(0)

    # Convertir 'VALOR' a float (sustituyendo coma por punto)
    serie_ventas['VGE'] = serie_ventas['VGE'].str.replace(',', '.', regex=False).astype(float)
    serie_rendimientos['RBT'] = serie_rendimientos['RBT'].str.replace(',', '.', regex=False).astype(float)

    return serie_ventas, serie_rendimientos

In [14]:
serie_ventas, serie_rendimientos = obtener_series_AEAT(url_aeat) 


Descarga cancelada. Se mantiene el archivo existente.
Archivo extraído: BdDVesge_extract\Diseño de registro tablas aux .xlsx
Archivo extraído: BdDVesge_extract\Vesge_pub_indexada.txt
Archivos .txt encontrados: ['Vesge_pub_indexada.txt']


In [15]:
serie_ventas.head()

,Fecha,VGE
0,1996-01-01,91.8399
1,1996-02-01,92.1908
2,1996-03-01,92.2892
3,1996-04-01,92.4449
4,1996-05-01,92.7558


In [16]:
serie_rendimientos.head()

,Fecha,RBT
0,1996-01-01,63.4698
1,1996-02-01,63.6296
2,1996-03-01,63.6336
3,1996-04-01,63.6187
4,1996-05-01,63.5993


In [17]:
serie_ventas.tail()

,Fecha,VGE
346,2024-11-01,131.7673
347,2024-12-01,130.7594
348,2025-01-01,133.0090
349,2025-02-01,134.5153
350,2025-03-01,134.1337


In [18]:
serie_rendimientos.tail()

,Fecha,RBT
346,2024-11-01,117.1724
347,2024-12-01,116.0921
348,2025-01-01,117.4045
349,2025-02-01,118.2299
350,2025-03-01,118.0460


#### Variables provenientes del Instituto Nacional de Estadística

Se obtienen cuatro series del INE:
* Índice de Producción Industrial. Índice general.
* Índice de Cifra de Negocios del Sector Servicios. Deflactado y CVEC
* Ocupación EPA. CVEC
* Producto Interior Bruto. CVEC

Para obtener estas series, se hace uso de la API disponible por el INE, que permite consultas sistemáticas con el código correspondiente para cada serie.

Los códigos de cada serie se encuentran en un fichero llamado "ine_codes".

In [ ]:
def obtener_ine_series():

    """
    Consulta la API del INE (Instituto Nacional de Estadística) para obtener series temporales
    utilizando los códigos proporcionados en un archivo Excel (`ine_codes.xlsx`).

    Retorna:
    -------
    pd.DataFrame
        DataFrame consolidado con una columna de fechas ('Fecha') y las columnas correspondientes
        a cada variable consultada, con sus respectivos valores temporales.
    """

    # Función interna que realiza la consulta al API del INE para un código dado
    def ine_request(ine_code):
        resultados = 999
        url = f"http://servicios.ine.es/wstempus/js/ES/DATOS_SERIE/{ine_code}?nult={resultados}"
        return requests.get(url).json()

    # Cargar el archivo con los códigos del INE y nombres de variables
    df_codigos = pd.read_excel('ine_codes.xlsx') 
    df_final = None 

    # Iterar por cada fila (serie) del archivo de códigos

    for _, fila in df_codigos.iterrows():
        codigo = fila['ine_code']
        nombre_variable = fila['Variable'].strip()

        try:
            # Realizar la consulta al API del INE
            datos = ine_request(codigo)

            # Inicializar listas para fechas y valores
            fecha_lista = []
            valor_lista = []

            # Extraer la información de cada observación
            for dato in datos['Data']:
                fecha = datetime.date.fromtimestamp(dato['Fecha'] // 1000)
                fecha_lista.append(fecha)
                valor_lista.append(dato['Valor'])

            # Crear DataFrame individual para esta serie
            df_variable = pd.DataFrame({
                'Fecha': fecha_lista,
                nombre_variable: valor_lista
            })

            # Convertir columna 'Literal' a datetime
            df_variable['Fecha'] = pd.to_datetime(df_variable['Fecha'], format="%Y-%m", errors='coerce') 

            # Si es la primera serie, iniciar df_final
            if df_final is None:
                df_final = df_variable
            else:
                # Unir esta nueva serie al DataFrame final por la columna 'Fecha'
                df_final = pd.merge(df_final, df_variable, on='Fecha', how='outer')

        except Exception as e:
            print(f"Error al procesar código {codigo} ({nombre_variable}): {e}")

    return df_final

In [20]:
series_ine = obtener_ine_series()

In [21]:
series_ine.head()

,Fecha,IPI,EPA,IASS,PIB
0,1992-01-01,95.971,NaN,NaN,NaN
1,1992-02-01,98.513,NaN,NaN,NaN
2,1992-03-01,95.526,NaN,NaN,NaN
3,1992-04-01,95.648,NaN,NaN,NaN
4,1992-05-01,94.328,NaN,NaN,NaN


In [22]:
series_ipi = series_ine[["Fecha", "IPI"]]

series_ipi.head()

series_pib = series_ine[["Fecha", "PIB"]]

primer_valido_pib = series_pib["PIB"].first_valid_index()
series_pib = series_pib .loc[primer_valido_pib:]

series_pib = series_pib.dropna().reset_index(drop=True)

In [23]:
series_epa = series_ine[["Fecha", "EPA"]]

primer_valido_epa = series_epa["EPA"].first_valid_index()
series_epa = series_epa.loc[primer_valido_epa:]

series_epa = series_epa.dropna().reset_index(drop=True)


In [24]:
series_iass = series_ine[["Fecha", "IASS"]]

primer_valido_iass = series_iass["IASS"].first_valid_index()
series_iass = series_iass.loc[primer_valido_iass:]

series_iass = series_iass.dropna().reset_index(drop=True)

series_iass.head()

,Fecha,IASS
0,2000-01-01,60.524
1,2000-02-01,60.300
2,2000-03-01,57.835
3,2000-04-01,62.108
4,2000-05-01,61.772


#### Variable exógena: Confianza del Consumidor

Se obtiene la serie del índice de confianza del consumidor mediante el paquete *eurostat*

In [ ]:
# Mediante el paquete eurostat, obtención de la serie 'CCI'

serie_confidence_indicator = eurostat.get_data('ei_bsco_m')

my_filter_pars = {'freq': 'M', 's_adj': 'SA', 'geo': 'ES', 'indic' : 'BS-CSMCI'}

serie_confidence_indicator_pre = eurostat.get_data_df('ei_bsco_m', filter_pars = my_filter_pars)

serie_confidence_indicator = serie_confidence_indicator_pre.melt(
    id_vars=['freq', 'indic', 's_adj', 'unit', 'geo\\TIME_PERIOD'],
    var_name='Fecha',
    value_name='CCI'
)

serie_confidence_indicator['Fecha'] = pd.to_datetime(serie_confidence_indicator['Fecha'], format="%Y-%m", errors='coerce') + pd.offsets.MonthBegin(0)

serie_confidence_indicator = serie_confidence_indicator[['Fecha', 'CCI']].dropna()

#### Importación de bienes total.

De manera mensual, el Ministerio de Economía, Comercio y Empresa, mediante DataComex (https://datacomex.comercio.es/), disponibiliza las importaciones mensuales, en euros.

Puesto que la Web no cuenta con una API o se permite la extracción mediante WebScrapping, se realiza la descarga a mano y se imputa de esta manera.

In [26]:
df_importaciones_raw = pd.read_csv('importaciones_input.csv')

In [27]:
# Se convierte la columna 'fila' en fechas (ej. 'Enero de 1995') → datetime (día 1 de mes)
df_importaciones_raw["Fecha"] = pd.to_datetime(df_importaciones_raw["fila"], format="%B de %Y", errors="coerce")

In [28]:
# Convertir la columna 'valor' de texto con coma decimal a float
df_importaciones_raw["valor"] = df_importaciones_raw["valor"].str.replace(",", "", regex=False) 
df_importaciones_raw["valor"] = df_importaciones_raw["valor"].astype("float")

In [29]:
serie_importaciones = df_importaciones_raw[["Fecha", "valor"]].rename(columns={"valor": "IMPB"})

In [30]:
serie_importaciones.dtypes

Fecha    datetime64[ns]
IMPB            float64
dtype: object

In [31]:
serie_importaciones

,Fecha,IMPB
0,1995-01-01,6.645390e+11
1,1995-02-01,6.965259e+11
2,1995-03-01,8.411816e+11
3,1995-04-01,6.849518e+11
4,1995-05-01,7.954167e+11
...,...,...
358,2024-11-01,3.777174e+12
359,2024-12-01,3.385983e+12
360,2025-01-01,3.597334e+12
361,2025-02-01,3.539794e+12


#### Base de datos de series de información coyuntural y económica (BDSICE) 

De manera mensual, el Ministerio de Economía, Comercio y Empresa, mediante la Dirección General de Análisis Económico, disponibiliza, entre otras, las tres siguientes series:

* Ocupados, desestacionalizados (en miles de personas)
* Afiliados, CVEC (en miles de personas)
* Importaciones a precios constantes (miles de euros)

In [ ]:
def leer_csv(ruta_archivo, col):
    """
    Lee un archivo CSV extraído de la BDSICE:
    - Codificado en UTF-16 LE (pero a menudo mal interpretado como UTF-8 con caracteres nulos)
    - Las dos primeras líneas son encabezados o metadatos
    - Datos a partir de la tercera línea, en formato: periodo;valor
    - El valor puede usar coma como separador decimal

    Parámetros:
    ----------
    ruta_archivo : str
        Ruta al archivo CSV a procesar.

    col : str
        Nombre de la columna para el valor extraído (por ejemplo: "EPA", "Tasa", etc.).

    Retorna:
    -------
    pd.DataFrame
        DataFrame con columnas:
        - 'Fecha': tipo datetime64, ajustado según frecuencia (mensual o trimestral).
        - 'col': valores numéricos extraídos del archivo.
    """
        
    try:
        # Leer binario, eliminar nulos, decodificar como UTF-8
        with open(ruta_archivo, "rb") as f:
            raw = f.read()
        text = raw.replace(b"\x00", b"").decode("utf-8", errors="ignore")

        # Separar líneas y saltar encabezados
        lines = text.splitlines()[2:]

        # Extraer datos válidos
        rows = []
        for line in lines:
            parts = line.strip().split(";")
            if len(parts) < 2:
                continue
            periodo = parts[0]
            match = re.match(r"(\d+,\d{1,3})", parts[1])
            if match:
                valor = float(match.group(1).replace(",", "."))
                rows.append((periodo, valor))

        # Crear DataFrame
        df = pd.DataFrame(rows, columns=["Periodo", col])
        df["Fecha"] = pd.to_datetime(df["Periodo"], format="%Y%m")

        if col in ["EPA"]:

            fecha_inicio = df["Fecha"].iloc[0]
            fechas_trimestre = [fecha_inicio + DateOffset(months=3*i) for i in range(len(df))]
            
            df["Fecha"] = fechas_trimestre
        
        return df[["Fecha", col]]

    except Exception as e:
        print(f"Error procesando {ruta_archivo}: {e}")
        return pd.DataFrame(columns=["Fecha", col])


In [33]:
serie_importaciones_n = leer_csv("Imports_BDSICE.csv", "IMPB")

serie_importaciones_n.head()


,Fecha,IMPB
0,1991-01-01,4863675.232
1,1991-02-01,5276385.152
2,1991-03-01,5091320.748
3,1991-04-01,5713493.789
4,1991-05-01,5107604.432


In [34]:
serie_importaciones_n.tail()

,Fecha,IMPB
406,2024-11-01,25771804.11
407,2024-12-01,23546412.93
408,2025-01-01,23843443.82
409,2025-02-01,25364471.31
410,2025-03-01,27193716.91


In [35]:
serie_afiliados_n = leer_csv("Afiliados_BDSICE.csv", "AFI")

serie_afiliados_n.head()

,Fecha,AFI
0,2008-01-01,19442.860
1,2008-02-01,19455.045
2,2008-03-01,19446.868
3,2008-04-01,19400.990
4,2008-05-01,19312.877


In [36]:
serie_afiliados_n.tail()

,Fecha,AFI
203,2024-12-01,21363.406
204,2025-01-01,21399.164
205,2025-02-01,21457.899
206,2025-03-01,21480.979
207,2025-04-01,21550.139


In [37]:
serie_epa_n = leer_csv("Ocupados_BDSICE.csv", "EPA")

serie_epa_n.head()

,Fecha,EPA
0,2005-01-01,18859.8
1,2005-04-01,19120.9
2,2005-07-01,19325.6
3,2005-10-01,19520.3
4,2006-01-01,19706.9


In [38]:
serie_epa_n.tail()

,Fecha,EPA
76,2024-01-01,21497.8
77,2024-04-01,21585.1
78,2024-07-01,21678.1
79,2024-10-01,21856.9
80,2025-01-01,22016.6


#### Creación de bases de datos

In [39]:
dfs = [serie_afiliados_n, serie_pmi, serie_ventas, serie_rendimientos, serie_importaciones_n, serie_confidence_indicator, series_iass, serie_epa_n, series_pib, series_ipi]

df_total = reduce(lambda left, right: pd.merge(left, right, on='Fecha', how='outer'), dfs)

# Ordenar por fecha
df_merged = df_total.sort_values('Fecha').reset_index(drop=True)


In [40]:
df_merged.columns

Index(['Fecha', 'AFI', 'PMI', 'VGE', 'RBT', 'IMPB', 'CCI', 'IASS', 'EPA',
       'PIB', 'IPI'],
      dtype='object')

In [41]:
df_merged.head()

,Fecha,AFI,PMI,VGE,RBT,IMPB,CCI,IASS,EPA,PIB,IPI
0,1986-06-01,NaN,NaN,NaN,NaN,NaN,-7.5,NaN,NaN,NaN,NaN
1,1986-07-01,NaN,NaN,NaN,NaN,NaN,-7.5,NaN,NaN,NaN,NaN
2,1986-08-01,NaN,NaN,NaN,NaN,NaN,-6.3,NaN,NaN,NaN,NaN
3,1986-09-01,NaN,NaN,NaN,NaN,NaN,-7.1,NaN,NaN,NaN,NaN
4,1986-10-01,NaN,NaN,NaN,NaN,NaN,-7.2,NaN,NaN,NaN,NaN


In [42]:
df_merged.tail()

,Fecha,AFI,PMI,VGE,RBT,IMPB,CCI,IASS,EPA,PIB,IPI
463,2025-01-01,21399.164,57.3,133.0090,117.4045,23843443.82,NaN,129.417,22016.6,121.9704,100.622
464,2025-02-01,21457.899,54.9,134.5153,118.2299,25364471.31,NaN,129.461,NaN,NaN,101.371
465,2025-03-01,21480.979,56.2,134.1337,118.0460,27193716.91,NaN,129.534,NaN,NaN,102.293
466,2025-04-01,21550.139,54.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
467,2025-05-01,NaN,53.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
df_merged.to_csv("Indicadores_Muestra.csv", index=False)

### Análisis Exploratorio de los Datos

In [ ]:
def eda_individual_dfs_guardar(df_list, output_dir="EDA_output"):
    """
    Realiza análisis exploratorio individual por serie temporal.

    Para cada DataFrame en la lista:
    - Calcula estadísticas básicas
    - Detecta valores faltantes
    - Extrae fechas de inicio y fin
    - Guarda gráficos de la serie temporal e histograma como PNG

    Parámetros:
    ----------
        df_list (list): Lista de DataFrames, cada uno debe tener una columna 'Fecha' + una o más series.
        output_dir (str): Nombre de la carpeta donde se guardarán los gráficos.

    Retorna:
    ----------
        DataFrame con resumen estadístico por serie.
    """

    # Crear carpeta de salida si no existe
    os.makedirs(output_dir, exist_ok=True)
    resumen = []

    for df in df_list:
        df = df.copy()
        df["Fecha"] = pd.to_datetime(df["Fecha"])
        df.set_index("Fecha", inplace=True)

        # Analizar cada columna de datos (excluyendo 'Fecha')
        for col in df.columns:
            serie = df[col].dropna()
            if serie.empty:
                continue  # Saltar si no hay datos

            # Estadísticas resumen
            resumen.append({
                "Variable": col,
                "Inicio": serie.index.min(),
                "Fin": serie.index.max(),
                "Media": serie.mean(),
                "Desviación típica": serie.std(),
                "Mínimo": serie.min(),
                "Q1": serie.quantile(0.25),
                "Mediana (Q2)": serie.median(),
                "Q3": serie.quantile(0.75),
                "Máximo": serie.max(),
                "N_muestras": serie.shape[0],
                "N_missing": df[col].isna().sum()
            })

            # Gráfico temporal (línea negra, sin título)
            plt.figure(figsize=(10, 4))
            plt.plot(serie.index, serie.values, color='black')
            plt.xlabel("Fecha")
            plt.ylabel(col)
            plt.xticks(rotation=0)
            plt.grid(True)
            plt.tight_layout()
            plt.savefig(os.path.join(output_dir, f"{col}_serie.png"))
            plt.close()

            # Histograma (negro, KDE activado)
            plt.figure(figsize=(6, 4))
            sns.histplot(serie, kde=True, bins=30, color='black')
            plt.xlabel(col)
            plt.xticks(rotation=0)
            plt.tight_layout()
            plt.savefig(os.path.join(output_dir, f"{col}_histograma.png"))
            plt.close()

    return pd.DataFrame(resumen)


In [45]:
resumen = eda_individual_dfs_guardar(dfs)

In [46]:
resumen

,Variable,Inicio,Fin,Media,Desviación típica,Mínimo,Q1,Mediana (Q2),Q3,Máximo,N_muestras,N_missing
0,AFI,2008-01-01,2025-04-01,1.847355e+04,1.444010e+03,1.623903e+04,1.736344e+04,1.836276e+04,1.939828e+04,2.155014e+04,208,0
1,PMI,2011-05-01,2025-05-01,5.203964e+01,6.828409e+00,7.100000e+00,5.020000e+01,5.410000e+01,5.620000e+01,6.250000e+01,169,0
2,VGE,1996-01-01,2025-03-01,1.184352e+02,1.275314e+01,8.341610e+01,1.096285e+02,1.186152e+02,1.278124e+02,1.456917e+02,351,0
3,RBT,1996-01-01,2025-03-01,9.139180e+01,1.495192e+01,6.346980e+01,8.008930e+01,9.930000e+01,1.005463e+02,1.182299e+02,351,0
4,IMPB,1991-01-01,2025-03-01,1.716177e+07,7.060477e+06,3.151780e+06,1.154381e+07,1.690168e+07,2.345482e+07,3.138979e+07,411,0
5,CCI,1986-06-01,2024-09-01,-1.275826e+01,9.627371e+00,-4.050000e+01,-2.015000e+01,-1.020000e+01,-5.200000e+00,2.500000e+00,460,0
6,IASS,2000-01-01,2025-03-01,8.770224e+01,1.654394e+01,5.700100e+01,7.635850e+01,8.272800e+01,9.590950e+01,1.295340e+02,303,0
7,EPA,2005-01-01,2025-01-01,1.929015e+04,1.281915e+03,1.710720e+04,1.839910e+04,1.928730e+04,2.025150e+04,2.201660e+04,81,0
8,PIB,1995-01-01,2025-01-01,9.667751e+01,1.394727e+01,6.701600e+01,8.805030e+01,1.000727e+02,1.054375e+02,1.219704e+02,121,0
9,IPI,1992-01-01,2025-03-01,1.059296e+02,1.165360e+01,6.796300e+01,9.851200e+01,1.018180e+02,1.170555e+02,1.326830e+02,399,0


In [47]:
resumen.to_csv("EDA_output/resumen_eda.csv", index=False)